In [ ]:
import tensorflow as tf

electron_model = tf.keras.models.load_model('/path/to/electron_model')
muon_model = tf.keras.models.load_model('/path/to/muon_model')
pi_plus_model = tf.keras.models.load_model('/path/to/pi_plus_model')
pi_minus_model = tf.keras.models.load_model('/path/to/pi_minus_model')
photon_model = tf.keras.models.load_model('/path/to/photon_model')
'''
EID = "/Users/wongdowling/Documents/Github/DQ_Dowling/new_ID/w_Ep/NNs/electronID_w_track_95"
MID = "/Users/wongdowling/Documents/Github/DQ_Dowling/new_ID/w_Ep/NNs/muonID_w_track_99"
PPID = "/Users/wongdowling/Documents/Github/DQ_Dowling/new_ID/w_Ep/NNs/pi+_ID_w_track_86"
PMID = "/Users/wongdowling/Documents/Github/DQ_Dowling/new_ID/w_Ep/NNs/pi-_ID_w_track_81"
PHID = "/Users/wongdowling/Documents/Github/DQ_Dowling/new_ID/w_Ep/NNs/photon_ID_w_track_89"
'''


In [ ]:
def multi_class_tagger(input_file):
    df = pd.read_csv(input_file)
    processed_data = df[["wid_x", "wid_y", "wew_x", "wew_y", "seed_x", "seed_y",
   "trkl_x", "trkl_y", "trkl_z", "trkl_px", "trkl_py", "trkl_pz","E/p",
   "h4_41", "h4_42", "h4_43", "h4_44", "h4_45", "h4_46"]].values
    # Get predictions from each model
    electron_prob = electron_model.predict(processed_data)
    muon_prob = muon_model.predict(processed_data)
    pi_plus_prob = pi_plus_model.predict(processed_data)
    pi_minus_prob = pi_minus_model.predict(processed_data)
    photon_prob = photon_model.predict(processed_data)

    # Combine predictions into a single array
    combined_probs = [electron_prob, muon_prob, pi_plus_prob, pi_minus_prob, photon_prob]

    # Determine the class with the highest probability
    final_class = np.argmax(combined_probs)
    return final_class

In [ ]:
import ROOT
import numpy as np

# Open the ROOT file
f = ROOT.TFile('muon_520_1000eve.root', 'UPDATE')
tree = f.Get('Events')

# Prepare a scalar Python variable to hold the data for the new branch
new_value = np.array([0], dtype=np.float32)  # Using an array of size 1 for compatibility

# Create a new branch. Use the 'new_value' array as the buffer
new_branch = tree.Branch('new_branch_name', new_value, 'new_branch_name/F')

# Generate or get your data
np_array = np.random.rand(1000).astype(np.float32)  # Example data

# Fill the branch with data from the NumPy array
for value in np_array:
    new_value[0] = value  # Update the buffer
    tree.Fill()

# Write the changes and close the file
tree.Write('', ROOT.TObject.kOverwrite)
f.Close()
